This script is necessary to identify the structures needed to compute the Climate Shelter Index (CSI), in particular:
- Surface Area of green spaces
- Presence of drinking fountains
- Presence of picnic tables and benches (rest area)

In addition the data about green area are combined with data about parks and gardens in Bologna

### 0. Libraries

In [ ]:
# libraries
import os
import geopandas as gpd
import shapely  #1.8.5 version 
from shapely.geometry import Polygon
from shapely.geometry import Point
import matplotlib
import mapclassify
import rasterio
import numpy as np # 1.23.5 version
import pandas as pd
import urllib.request
import pyrosm

### 1. Surface Area 
Calculate the surface area of each green space in the municipality of Bologna

In [2]:
# Load the shapefile of green area 
ar_vrd = gpd.read_file(r"D:\Bologna_DBSN\Bologna_dbsn_selected\ar_vrd.shp")

In [3]:
# check the current CRS
ar_vrd.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
target_crs = "EPSG:32633"
# Reproject the GeoDataFrame to the target CRS
ar_vrd = ar_vrd.to_crs(target_crs)

In [6]:
# Calculate Surface Area in meters an hectares
ar_vrd['Area_SqMeters'] = ar_vrd.geometry.area
ar_vrd['Area_Ha'] = ar_vrd['Area_SqMeters'] / 10000

In [ ]:
# Reproject the GeoDataFrame to the orignal CRS
ar_vrd= ar_vrd.to_crs(epsg=4326)

# Save the GeoDataFrame with the surface area as a shapefile
ar_vrd.to_file(r'D:\Climate_Shelter_Index\ar_vrd_surface.shp')

### 2. Drinking fountains
Load data about drinking fountains

In [ ]:
url_download_bologna_pbf = 'https://osmit-estratti.wmcloud.org/dati/poly/comuni/pbf/037006_Bologna.osm.pbf'
urllib.request.urlretrieve(url_download_bologna_pbf ,"bologna_osm.pbf")
osm = pyrosm.OSM("bologna_osm.pbf") 

In [ ]:
# Identify drinking fountains using some filters
custom_filter = {'amenity': ['drinking_water']}
drinking_fountain = osm.get_pois(custom_filter=custom_filter)

In [ ]:
# Create a GeoDataFrame for drinking fountains
drinking_fountain_gdf = gpd.GeoDataFrame(drinking_fountain, geometry=[Point(x, y) for x, y in zip(drinking_fountain["lon"], drinking_fountain["lat"])])

In [ ]:
# Save the GeoDataFrame as a shapefile
drinking_fountain_gdf.to_file('D:\Climate_Shelter_Index\drinking_fountains.shp')

#### Green areas + Drinking fountains
Check if there are drinking fountains in the green areas

In [15]:
# Load the shapefiles
green_area = gpd.read_file(r'D:\Climate_Shelter_Index\ar_vrd_surface.shp')
fountains = gpd.read_file(r"D:\Climate_Shelter_Index\drinking_fountains.shp")

In [ ]:
# Perform a spatial join to check if each park contains a fountain
ga_with_fountains = gpd.sjoin(green_area, fountains, how="left", op="intersects")

# Create a new column 'drinking_fountain' indicating if there is a fountain
ga_with_fountains["drinking_fountain"] = ga_with_fountains["index_right"].notnull().replace({True: "yes", False: "no"})

In [18]:
# Create a new column 'id_fountain' with the ID of the drinking fountain if present
ga_with_fountains["id_fountain"] = ga_with_fountains["id"].fillna("N/A")

In [14]:
# Check all the columns
ga_with_fountains.columns

Index(['OBJECTID', 'ar_vrd_ty', 'scril', 'classid', 'meta_ist', 'shape_Leng',
       'shape_Area', 'Area_SqMet', 'Area_Ha', 'geometry', 'index_right',
       'version', 'id', 'lat', 'tags', 'changeset', 'timestamp', 'lon', 'name',
       'amenity', 'fountain', 'source', 'osm_type', 'drinking_fountain',
       'id_fountain'],
      dtype='object')

In [20]:
# Select the columns you want to keep in the final GeoDataFrame
final_columns = ["OBJECTID", "classid", "shape_Leng", "shape_Area", "Area_SqMet", "Area_Ha", "geometry", 'amenity',
                 'drinking_fountain' ,'id_fountain']


In [ ]:
# Create a new GeoDataFrame with the selected columns
final_gdf = ga_with_fountains[final_columns]

final_gdf["id_fountain"] =  final_gdf["id_fountain"].apply(lambda x: str(x).replace('.0', '') if x != 'N/A' else x)

final_gdf[final_gdf['drinking_fountain']=='yes'].head(3)


In [ ]:
# Rename the column
final_gdf.rename(columns={"Area_SqMet": "Area_SqM",
                          "drinking_fountain":"d_fountain",
                          "id_fountain":"id_df"
                          }, 
                 inplace=True)

In [23]:
# Save the final GeoDataFrame as a shapefile
final_gdf.to_file(r"D:\Climate_Shelter_Index\ar_vrd_surface_drinkingfount.shp")


### 3. Picnic tables
Load data about picnic tables

In [ ]:
# Identify picnic tables using some filters
custom_filter = {'leisure': ['picnic_table']}
picnic_table = osm.get_pois(custom_filter=custom_filter)

In [12]:
# Create a GeoDataFrame for picnic tables
picnic_table_gdf = gpd.GeoDataFrame(picnic_table, geometry=[Point(x, y) for x, y in zip(picnic_table["lon"], picnic_table["lat"])])

In [15]:
# Save the GeoDataFrame as a shapefile
picnic_table_gdf.to_file('D:\Climate_Shelter_Index\picnic_tables.shp')

#### Green areas + Drinking fountains + Picnic tables
Check if there are picnic tables in the green areas

In [24]:
# Load the shapefiles
ar_vrd_surface_drinkingfount = gpd.read_file(r"D:\Climate_Shelter_Index\ar_vrd_surface_drinkingfount.shp")
picnic_tables = gpd.read_file(r'D:\Climate_Shelter_Index\picnic_tables.shp')

In [ ]:
# Perform a spatial join to check if each park contains a picnic table
ga_pt = gpd.sjoin(ar_vrd_surface_drinkingfount, picnic_tables, how="left", op="intersects")

# Create a new column 'picnic_table' indicating if there is a picnic table
ga_pt["picnic_table"] = ga_pt["index_right"].notnull().replace({True: "yes", False: "no"})

In [28]:
# Create a new column 'id_pt' with the ID of the picnic table if present
ga_pt["id_pt"] = ga_pt["id"].fillna("N/A")

In [ ]:
final_columns = ['OBJECTID', 'classid', 'shape_Leng', 'shape_Area', 'Area_SqM',
        'Area_Ha', 'amenity', 'd_fountain', 'id_df', 'geometry','picnic_table', 'id_pt']
# Create a new GeoDataFrame with the selected columns
final_gdf2 = ga_pt[final_columns]

final_gdf2['id_pt'] = final_gdf2['id_pt'].apply(lambda x: str(x).replace('.0', '') if x != 'N/A' else x)


In [ ]:
# Save the final GeoDataFrame as a shapefile
final_gdf2.to_file(r"D:\Climate_Shelter_Index\ar_vrd_s_df_pt.shp")

### 4. Benches
Load data about benches

In [ ]:
# Identify benches using some filters
custom_filter = {'amenity': ['bench']}
benches = osm.get_pois(custom_filter=custom_filter)

In [7]:
# Create a GeoDataFrame for benches
benches_gdf = gpd.GeoDataFrame(benches, geometry=[Point(x, y) for x, y in zip(benches["lon"], benches["lat"])])

In [9]:
# Save the GeoDataFrame as a shapefile
benches_gdf.to_file(r'D:\Climate_Shelter_Index\benches.shp')

#### Green areas + Drinking fountains + Picnic tables + Benches
Check if there are benches in the green areas

In [10]:
# Load shapefiles
ar_vrd_s_df_pt = gpd.read_file(r"D:\Climate_Shelter_Index\ar_vrd_s_df_pt.shp")
benches = gpd.read_file(r'D:\Climate_Shelter_Index\benches.shp')

In [ ]:
# Perform a spatial join to check if each park contains benches
ga_benches = gpd.sjoin(ar_vrd_s_df_pt, benches, how="left", op="intersects")

# Create a new column 'benches' indicating if there is a bench
ga_benches["benches"] = ga_benches["index_right"].notnull().replace({True: "yes", False: "no"})

In [13]:
# Create a new column 'id_bench' with the ID of the bench if present
ga_benches["id_bench"] = ga_benches["id"].fillna("N/A")

In [18]:
ga_benches.columns

Index(['OBJECTID', 'classid', 'shape_Leng', 'shape_Area', 'Area_SqM',
       'Area_Ha', 'amenity_left', 'd_fountain', 'id_df', 'picnic_tab', 'id_pt',
       'geometry', 'index_right', 'id', 'version', 'lat', 'changeset', 'lon',
       'timestamp', 'tags', 'amenity_right', 'osm_type', 'benches',
       'id_bench'],
      dtype='object')

In [ ]:
final_columns = ['OBJECTID', 'classid', 'shape_Leng', 'shape_Area', 'Area_SqM',
        'Area_Ha',  'd_fountain', 'id_df', 'geometry','picnic_tab', 'id_pt',
        'benches', 'id_bench']
# Create a new GeoDataFrame with the selected columns
final_gdf3 = ga_benches[final_columns]

final_gdf3['id_bench'] = final_gdf3['id_bench'].apply(lambda x: str(x).replace('.0', '') if x != 'N/A' else x)


In [ ]:
final_gdf3['OBJECTID'] = final_gdf3['OBJECTID'].apply(lambda x: str(x).replace('.0', '') if x != 'N/A' else x)

In [31]:
# Save the final GeoDataFrame as a shapefile
final_gdf3.to_file(r"D:\Climate_Shelter_Index\ar_vrd_s_df_pt_b.shp")

### 5. Park names
Load data about garden and park in Bologna (with their names)

In [ ]:
#Load shapefile
ar_vrd_s_df_pt_b = gpd.read_file(r"D:\Climate_Shelter_Index\ar_vrd_s_df_pt_b.shp")
park_names = gpd.read_file(r"D:\Climate_Shelter_Index\carta-tecnica-comunale-toponimi-parchi-e-giardini\carta-tecnica-comunale-toponimi-parchi-e-giardini.shp")

In [ ]:
# Perform a spatial join 
final_gdf4 = gpd.sjoin(ar_vrd_s_df_pt_b, park_names, how="left", op="intersects")

In [6]:
final_gdf4['index_right'] = final_gdf4['index_right'].apply(lambda x: str(x).replace('.0', '') if x != 'N/A' else x)

In [12]:
# Rename the column
final_gdf4.rename(columns={"index_right": "id_park",}, 
                 inplace=True)

In [13]:
# Save the final GeoDataFrame as a shapefile
final_gdf4.to_file(r"D:\Climate_Shelter_Index\ar_vrd_s_df_pt_b_names.shp")